# 프로세스 준비

In [1]:
import pandas as pd
import warnings, os, shutil
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
warnings.simplefilter(action='ignore')
pd.options.display.max_columns = None

## 데이터 확인

In [3]:
dataDir = "data path here"
S2DataDir = "Setinel data path here"
outputDir = "result data"

files_list_data = os.listdir(dataDir)
files_list_STNL = os.listdir(S2DataDir)

files_list_data = [file for file in files_list_data if file.endswith('.xlsx')]
files_list_STNL = [file for file in files_list_STNL if file.endswith('.csv')]

In [ ]:
print('data file list :', files_list_data)
print(f"data file : {len(files_list_data)}개")
print('='*100)
print('S2 file list :', files_list_STNL)
print(f"Sentinel-2 file : {len(files_list_STNL)}개")

In [ ]:
loc_list = list(map(lambda x: x.strip('.xlsx'), files_list_data))
files_list_STNL_matched = []

for location_data in loc_list:
    for location_S2 in files_list_STNL:
        if location_data in location_S2:
            files_list_STNL_matched.append(location_S2)

print(f"Location 목록 : {loc_list}")
print(f"조인 가능 데이터 목록 : {files_list_STNL_matched}")

## data 데이터 로드

In [6]:
''' [Location명]_data 변수 이름으로 데이터프레임 생성 '''
for file in files_list_data:
    globals()[f'{file.strip(".xlsx")}_data'] = pd.read_excel(f"{dataDir}\\{file}")

## 센티넬 데이터 로드

In [7]:
''' [Location명]_S2 변수 이름으로 데이터프레임 생성 '''
for file in files_list_STNL_matched:
    globals()[f"{file.strip('.csv')}"] = pd.read_csv(f"{S2DataDir}\\{file}", encoding='utf-8')

---

# 데이터 전처리

## data 데이터 전처리 및 센티넬 데이터와 조인

In [ ]:
''' 폴더 생성 '''
if os.path.exists(outputDir):
    shutil.rmtree(outputDir)
os.makedirs(outputDir)


for location in loc_list:
    
    ''' data 데이터 전처리 '''
    df_data = globals()[f'{location}_data']
    df_data = df_data.drop(columns="Unnamed: 0")
    df_data.dropna(inplace=True)
    df_data[['Year', 'Month', 'Date']] = df_data[['Year', 'Month', 'Date']].astype('uint64').astype('string')
    df_data.rename(columns={'Date':'Day'}, inplace=True)

    datePadding = lambda x: '0' + x if len(x) == 1 else x
    df_data['Month'] = df_data['Month'].apply(datePadding)
    df_data['Day'] = df_data['Day'].apply(datePadding)
    df_data['DATE'] = df_data['Year'] + '-' + df_data['Month'] + '-' + df_data['Day']
    df_data['DATE'] = df_data['DATE'].astype('datetime64')
    
    try:
        df_data = df_data[['DATE', 'Ref_BLU', 'Ref_GRN', 'Ref_RED', 'Ref_NIR', 'Ref_PAR', 'NDVI']]
    except:
        df_data = df_data[['DATE', 'Ref_BLU', 'Ref_GRN', 'Ref_RED', 'Ref_NIR', 'NDVI']]

    df_data.set_index('DATE', inplace=True)


    ''' 센티넬 데이터 조인 준비 '''
    df_S2 = globals()[f'{location}_S2']
    df_S2['DATE'] = df_S2['DATE'].astype('datetime64')

    cols_by_buff_dict = { '5M': {'Blue':'BLUE_5M', 'Green':'GREEN_5M', 'Red':'RED_5M', 'NIR':'NIR_5M', 'NDVI':'NDVI_5M'},
                         '10M': {'Blue':'BLUE_10M', 'Green':'GREEN_10M', 'Red':'RED_10M', 'NIR':'NIR_10M', 'NDVI':'NDVI_10M'},
                         '15M': {'Blue':'BLUE_15M', 'Green':'GREEN_15M', 'Red':'RED_15M', 'NIR':'NIR_15M', 'NDVI':'NDVI_15M'},
                         '20M': {'Blue':'BLUE_20M', 'Green':'GREEN_20M', 'Red':'RED_20M', 'NIR':'NIR_20M', 'NDVI':'NDVI_20M'},
                         '25M': {'Blue':'BLUE_25M', 'Green':'GREEN_25M', 'Red':'RED_25M', 'NIR':'NIR_25M', 'NDVI':'NDVI_25M'},
                         '30M': {'Blue':'BLUE_30M', 'Green':'GREEN_30M', 'Red':'RED_30M', 'NIR':'NIR_30M', 'NDVI':'NDVI_30M'}}
    
    df_5M = df_S2[df_S2['BUFF_DIST'] == 5][['DATE', 'Blue', 'Green', 'Red', 'NIR', 'NDVI']].rename(columns=cols_by_buff_dict['5M']).groupby('DATE').mean()
    df_10M = df_S2[df_S2['BUFF_DIST'] == 10][['DATE', 'Blue', 'Green', 'Red', 'NIR', 'NDVI']].rename(columns=cols_by_buff_dict['10M']).groupby('DATE').mean()
    df_15M = df_S2[df_S2['BUFF_DIST'] == 15][['DATE', 'Blue', 'Green', 'Red', 'NIR', 'NDVI']].rename(columns=cols_by_buff_dict['15M']).groupby('DATE').mean()
    df_20M = df_S2[df_S2['BUFF_DIST'] == 20][['DATE', 'Blue', 'Green', 'Red', 'NIR', 'NDVI']].rename(columns=cols_by_buff_dict['20M']).groupby('DATE').mean()
    df_25M = df_S2[df_S2['BUFF_DIST'] == 25][['DATE', 'Blue', 'Green', 'Red', 'NIR', 'NDVI']].rename(columns=cols_by_buff_dict['25M']).groupby('DATE').mean()
    df_30M = df_S2[df_S2['BUFF_DIST'] == 30][['DATE', 'Blue', 'Green', 'Red', 'NIR', 'NDVI']].rename(columns=cols_by_buff_dict['30M']).groupby('DATE').mean()


    ''' 조인 결과 저장 '''
    result_df = pd.concat([df_data, df_5M, df_10M, df_15M, df_20M, df_25M, df_30M], axis=1, join='inner')
    result_df.reset_index(inplace=True)
    result_df['DATE'] = result_df['DATE'].astype('string')
    result_df.to_excel(f'{outputDir}\\{location}_total.xlsx', engine='xlsxwriter', index=False)
    print(f'{outputDir}\\{location}_total.xlsx 저장 완료.')
    globals()[f'{location}_total'] = result_df.copy()

---

# 통계 데이터 생성

## 통계

### 평균

In [9]:
bands_cols_dict = { 'blue':{'Ref_BLU':'Field', 'BLUE_5M':'5M', 'BLUE_10M':'10M', 'BLUE_15M':'15M', 'BLUE_20M':'20M', 'BLUE_25M':'25M', 'BLUE_30M':'30M'},
                   'green':{'Ref_GRN':'Field', 'GREEN_5M':'5M', 'GREEN_10M':'10M', 'GREEN_15M':'15M', 'GREEN_20M':'20M', 'GREEN_25M':'25M', 'GREEN_30M':'30M'},
                     'red':{'Ref_RED':'Field', 'RED_5M':'5M', 'RED_10M':'10M', 'RED_15M':'15M', 'RED_20M':'20M', 'RED_25M':'25M', 'RED_30M':'30M'},
                     'nir':{'Ref_NIR':'Field', 'NIR_5M':'5M', 'NIR_10M':'10M', 'NIR_15M':'15M', 'NIR_20M':'20M', 'NIR_25M':'25M', 'NIR_30M':'30M'},
                    'ndvi':{'NDVI':'Field', 'NDVI_5M':'5M', 'NDVI_10M':'10M', 'NDVI_15M':'15M', 'NDVI_20M':'20M', 'NDVI_25M':'25M', 'NDVI_30M':'30M'}}

In [10]:
for Location in loc_list:    
    
    target_df = globals()[f'{Location}_total']
    
    BLUE_mean = pd.DataFrame(target_df[['BLUE_5M', 'BLUE_10M', 'BLUE_15M', 'BLUE_20M', 'BLUE_25M', 'BLUE_30M', 'Ref_BLU']].mean())
    BLUE_mean.rename(index=bands_cols_dict['blue'], inplace=True)
    
    GREEN_mean = pd.DataFrame(target_df[['GREEN_5M', 'GREEN_10M', 'GREEN_15M', 'GREEN_20M', 'GREEN_25M', 'GREEN_30M', 'Ref_GRN']].mean())
    GREEN_mean.rename(index=bands_cols_dict['green'], inplace=True)
    
    RED_mean = pd.DataFrame(target_df[['RED_5M', 'RED_10M', 'RED_15M', 'RED_20M', 'RED_25M', 'RED_30M', 'Ref_RED']].mean())
    RED_mean.rename(index=bands_cols_dict['red'], inplace=True)
    
    NIR_mean = pd.DataFrame(target_df[['NIR_5M', 'NIR_10M', 'NIR_15M', 'NIR_20M', 'NIR_25M', 'NIR_30M', 'Ref_NIR']].mean())
    NIR_mean.rename(index=bands_cols_dict['nir'], inplace=True)
    
    NDVI_mean = pd.DataFrame(target_df[['NDVI_5M', 'NDVI_10M', 'NDVI_15M', 'NDVI_20M', 'NDVI_25M', 'NDVI_30M', 'NDVI']].mean())
    NDVI_mean.rename(index=bands_cols_dict['ndvi'], inplace=True)

    mean_df = pd.concat([BLUE_mean, GREEN_mean, RED_mean, NIR_mean, NDVI_mean], axis=1)
    mean_df.columns = ['BLUE', 'GREEN', 'RED', 'NIR', 'NDVI']
    globals()[f'{Location}_mean'] = mean_df.copy()

### 표준편차

In [11]:
for Location in loc_list:
    
    target_df = globals()[f'{Location}_total']
    
    BLUE_std = pd.DataFrame(target_df[['BLUE_5M', 'BLUE_10M', 'BLUE_15M', 'BLUE_20M', 'BLUE_25M', 'BLUE_30M', 'Ref_BLU']].std())
    BLUE_std.rename(index=bands_cols_dict['blue'], inplace=True)
    
    GREEN_std = pd.DataFrame(target_df[['GREEN_5M', 'GREEN_10M', 'GREEN_15M', 'GREEN_20M', 'GREEN_25M', 'GREEN_30M', 'Ref_GRN']].std())
    GREEN_std.rename(index=bands_cols_dict['green'], inplace=True)
    
    RED_std = pd.DataFrame(target_df[['RED_5M', 'RED_10M', 'RED_15M', 'RED_20M', 'RED_25M', 'RED_30M', 'Ref_RED']].std())
    RED_std.rename(index=bands_cols_dict['red'], inplace=True)
    
    NIR_std = pd.DataFrame(target_df[['NIR_5M', 'NIR_10M', 'NIR_15M', 'NIR_20M', 'NIR_25M', 'NIR_30M', 'Ref_NIR']].std())
    NIR_std.rename(index=bands_cols_dict['nir'], inplace=True)
    
    NDVI_std = pd.DataFrame(target_df[['NDVI_5M', 'NDVI_10M', 'NDVI_15M', 'NDVI_20M', 'NDVI_25M', 'NDVI_30M', 'NDVI']].std())
    NDVI_std.rename(index=bands_cols_dict['ndvi'], inplace=True)

    std_df = pd.concat([BLUE_std, GREEN_std, RED_std, NIR_std, NDVI_std], axis=1)
    std_df.columns = ['BLUE', 'GREEN', 'RED', 'NIR', 'NDVI']
    globals()[f'{Location}_std'] = std_df.copy()

### 오차
- ( 버퍼별 평균 - 필드 평균 )
- 4자리에서 반올림한 백분율로 표기

In [12]:
for location in loc_list:
    diff = globals()[f'{location}_mean']
    globals()[f'{location}_diff'] = ((diff.loc[['5M', '10M', '15M', '20M', '25M', '30M'], :] - diff.loc['Field', :]) * 100)
    globals()[f'{location}_diff'] = globals()[f'{location}_diff'].round(3).astype('string') + '%'

## 회귀분석 결과

### R-Squared

In [13]:
def df_to_R2_list(train_data, target_data, result_list):
    X = train_data.to_numpy().reshape(-1, 1)
    Y = target_data.to_numpy().reshape(-1, 1)

    green_model = LinearRegression()
    green_model.fit(X, Y)
    pred = green_model.predict(X)

    r2 = r2_score(Y, pred)
    result_list.append(r2)
    return result_list

In [14]:
for location in loc_list:
    
    buff_list = list(range(5, 31, 5))
    blue_r2_list = []
    green_r2_list = []
    red_r2_list = []
    nir_r2_list = []
    ndvi_r2_list = []
    
    target_df = globals()[f'{location}_total']
    
    for buff in buff_list:
        df_to_R2_list(target_df[f'BLUE_{buff}M'], target_df['Ref_BLU'], blue_r2_list)
        df_to_R2_list(target_df[f'GREEN_{buff}M'], target_df['Ref_GRN'], green_r2_list)
        df_to_R2_list(target_df[f'RED_{buff}M'], target_df['Ref_RED'], red_r2_list)
        df_to_R2_list(target_df[f'NIR_{buff}M'], target_df['Ref_NIR'], nir_r2_list)
        df_to_R2_list(target_df[f'NDVI_{buff}M'], target_df['NDVI'], ndvi_r2_list)

    globals()[f'{location}_R2'] = pd.DataFrame({'Buffer(M)':buff_list, 'R2_BLUE':blue_r2_list, 'R2_GREEN':green_r2_list,
                                                'R2_RED':red_r2_list, 'R2_NIR':nir_r2_list, 'R2_NDVI':ndvi_r2_list}).set_index('Buffer(M)')

### Coefficient

### RMSD ( Root Mean Sqaured Deviation )

In [15]:
for location in loc_list:
    
    buff_list = list(range(5, 31, 5))
    blue_RMSD_list = []
    green_RMSD_list = []
    red_RMSD_list = []
    nir_RMSD_list = []
    ndvi_RMSD_list = []
    
    target_df = globals()[f'{location}_total']
    
    for buff in buff_list:

        blue_RMSD = mean_squared_error(target_df[f'BLUE_{buff}M'], target_df['Ref_BLU'], squared=False)
        blue_RMSD_list.append(blue_RMSD)

        green_RMSD = mean_squared_error(target_df[f'GREEN_{buff}M'], target_df['Ref_GRN'], squared=False)
        green_RMSD_list.append(green_RMSD)

        red_RMSD = mean_squared_error(target_df[f'RED_{buff}M'], target_df['Ref_RED'], squared=False)
        red_RMSD_list.append(red_RMSD)

        nir_RMSD2 = mean_squared_error(target_df[f'NIR_{buff}M'], target_df['Ref_NIR'], squared=False)
        nir_RMSD_list.append(nir_RMSD2)

        ndvi_RMSD = mean_squared_error(target_df[f'NDVI_{buff}M'], target_df['NDVI'], squared=False)
        ndvi_RMSD_list.append(ndvi_RMSD)

    globals()[f'{location}_RMSD'] = pd.DataFrame({'Buffer(M)':buff_list, 'RMSD_BLUE':blue_RMSD_list, 'RMSD_GREEN':green_RMSD_list,
                                                'RMSD_RED':red_RMSD_list, 'RMSD_NIR':nir_RMSD_list, 'RMSD_NDVI':ndvi_RMSD_list}).set_index('Buffer(M)')

---

# 데이터 조회

## 평균

In [16]:
location = widgets.Dropdown(options=loc_list)

@interact
def showMeanByLocation(Location = location):
    return globals()[f'{Location}_mean']

interactive(children=(Dropdown(description='Location', options=('AMD', 'GCK', 'HAWS1', 'JJ', 'PYC'), value='AM…

In [17]:
location2 = widgets.Dropdown(options=loc_list)

@interact
def showMeanGraphByLocation(Location = location2):
    data = globals()[f'{Location}_mean']
    data = data.T.loc[['BLUE', 'GREEN', 'RED', 'NIR'], :]
    
    plt.figure(figsize=(9, 7))
    
    buff_5M = plt.plot(data['5M'], color='blue', label='5M')
    buff_10M = plt.plot(data['10M'], color='green', label='10M')
    buff_15M = plt.plot(data['15M'], color='red', label='15M')
    buff_20M = plt.plot(data['20M'], color='magenta', label='20M')
    buff_25M = plt.plot(data['25M'], color='cyan', label='25M')
    buff_30M = plt.plot(data['30M'], color='skyblue', label='30M')
    field = plt.plot(data['Field'], color='grey', label='Field', marker='*')
    
    plt.xlabel('Buffer', fontsize=13)
    plt.ylabel('Value', rotation=0, labelpad=18, fontsize=13)
    plt.ylim(0, data.max().max() * 1.25)
    plt.title(f'{Location} Mean', fontsize=18)
    plt.grid(axis='y')
    
    lines = buff_5M + buff_10M + buff_15M + buff_20M + buff_25M + buff_30M + field
    labels = [line.get_label() for line in lines]
    plt.legend(lines, labels)
    
    plt.show()

interactive(children=(Dropdown(description='Location', options=('AMD', 'GCK', 'HAWS1', 'JJ', 'PYC'), value='AM…

## 표준편차

In [18]:
location3 = widgets.Dropdown(options=loc_list)

@interact
def showSTDByLocation(Location = location3):
    return globals()[f'{Location}_std']

interactive(children=(Dropdown(description='Location', options=('AMD', 'GCK', 'HAWS1', 'JJ', 'PYC'), value='AM…

## 오차

In [19]:
location4 = widgets.Dropdown(options=loc_list)

@interact
def showMeanByLocation(Location = location4):
    return globals()[f'{Location}_diff']

interactive(children=(Dropdown(description='Location', options=('AMD', 'GCK', 'HAWS1', 'JJ', 'PYC'), value='AM…

## R-Squared

In [20]:
location5 = widgets.Dropdown(options=loc_list)

@interact
def showR2ByLocation(Location = location5):
    return globals()[f'{Location}_R2']

interactive(children=(Dropdown(description='Location', options=('AMD', 'GCK', 'HAWS1', 'JJ', 'PYC'), value='AM…

## Coefficient

## RMSD

In [21]:
location7 = widgets.Dropdown(options=loc_list)

@interact
def showRMSDByLocation(Location = location7):
    return globals()[f'{Location}_RMSD']

interactive(children=(Dropdown(description='Location', options=('AMD', 'GCK', 'HAWS1', 'JJ', 'PYC'), value='AM…